In [1]:
import numpy as np
import pandas as pd
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
import pretty_midi
import warnings
import os

In [3]:
def get_genres(path):
    """
    This function reads the genre labels and puts it into a pandas DataFrame.
    
    @input path: The path to the genre label file.
    @type path: String
    
    @return: A pandas dataframe containing the genres and midi IDs.
    @rtype: pandas.DataFrame
    """
    ids = []
    genres = []
    with open(path) as f:
        line = f.readline()
        while line:
            if line[0] != '#':
                [x, y, *_] = line.strip().split("\t")
                ids.append(x)
                genres.append(y)
            line = f.readline()
    genre_df = pd.DataFrame(data={"Genre": genres, "TrackID": ids})
    return genre_df

# Get the Genre DataFrame
genre_path = "msd_tagtraum_cd1.cls"
genre_df = get_genres(genre_path)

# Create Genre List and Dictionary
label_list = list(set(genre_df.Genre))
label_dict = {lbl: label_list.index(lbl) for lbl in label_list}

# Print to Visualize
print(genre_df.head(), end="\n\n")
print(label_list, end="\n\n")
print(label_dict, end="\n\n")

      Genre             TrackID
0  Pop_Rock  TRAAAAK128F9318786
1       Rap  TRAAAAW128F429D538
2  Pop_Rock  TRAAABD128F429CF47
3      Jazz  TRAAAED128E0783FAB
4  Pop_Rock  TRAAAEF128F4273421

['RnB', 'Pop_Rock', 'Blues', 'International', 'Reggae', 'Jazz', 'New Age', 'Folk', 'Latin', 'Vocal', 'Electronic', 'Country', 'Rap']

{'RnB': 0, 'Pop_Rock': 1, 'Blues': 2, 'International': 3, 'Reggae': 4, 'Jazz': 5, 'New Age': 6, 'Folk': 7, 'Latin': 8, 'Vocal': 9, 'Electronic': 10, 'Country': 11, 'Rap': 12}



In [5]:
def get_matched_midi(midi_folder, genre_df):
    """
    This function loads in midi file paths that are found in the given folder, puts this data into a
    pandas DataFrame, then matches each entry with a genre described in get_genres.
    
    @input midi_folder: The path to the midi files.
    @type midi_folder: String
    @input genre_df: The genre label dataframe generated by get_genres.
    @type genre_df: pandas.DataFrame
    
    @return: A dataframe of track id and path to a midi file with that track id.
    @rtype: pandas.DataFrame
    """
    # Get All Midi Files
    track_ids, file_paths = [], []
    for dir_name, subdir_list, file_list in os.walk(midi_folder):
        if len(dir_name) == 36:
            track_id = dir_name[18:]
            file_path_list = ["/".join([dir_name, file]) for file in file_list]
            for file_path in file_path_list:
                track_ids.append(track_id)
                file_paths.append(file_path)
    all_midi_df = pd.DataFrame({"TrackID": track_ids, "Path": file_paths})
    
    # Inner Join with Genre Dataframe
    df = pd.merge(all_midi_df, genre_df, on='TrackID', how='inner')
    return df.drop(["TrackID"], axis=1)

# Obtain DataFrame with Matched Genres to File Paths
midi_path = "lmd_matched"
matched_midi_df = get_matched_midi(midi_path, genre_df)

# Print to Check Correctness
print(matched_midi_df.head())

                                                Path     Genre
0  lmd_matched/R/R/U/TRRRUFD12903CD7092/6c460e4c5...  Pop_Rock
1  lmd_matched/R/R/U/TRRRUFD12903CD7092/6ca2a1f03...  Pop_Rock
2  lmd_matched/R/R/A/TRRRAJP128E0793859/2c78d25cb...       RnB
3  lmd_matched/R/R/F/TRRRFLX128F9326186/afd3595e5...  Pop_Rock
4  lmd_matched/R/R/F/TRRRFLX128F9326186/74582fd38...  Pop_Rock


In [6]:
%%time
def normalize_features(features):
    """
    This function normalizes the features to the range [-1, 1]
    
    @input features: The array of features.
    @type features: List of float
    
    @return: Normalized features.
    @rtype: List of float
    """
    tempo = (features[0] - 150) / 300
    num_sig_changes = (features[1] - 2) / 10
    resolution = (features[2] - 260) / 400
    time_sig_1 = (features[3] - 3) / 8
    time_sig_2 = (features[4] - 3) / 8
    return [tempo, resolution, time_sig_1, time_sig_2]


def get_features(path):
    """
    This function extracts the features from a midi file when given its path.
    
    @input path: The path to the midi file.
    @type path: String
    
    @return: The extracted features.
    @rtype: List of float
    """
    try:
        # Test for Corrupted Midi Files
        with warnings.catch_warnings():
            warnings.simplefilter("error")
            file = pretty_midi.PrettyMIDI(path)
            
            tempo = file.estimate_tempo()
            num_sig_changes = len(file.time_signature_changes)
            resolution = file.resolution
            ts_changes = file.time_signature_changes
            ts_1 = 4
            ts_2 = 4
            if len(ts_changes) > 0:
                ts_1 = ts_changes[0].numerator
                ts_2 = ts_changes[0].denominator
            return normalize_features([tempo, num_sig_changes, resolution, ts_1, ts_2])
    except:
        return None


def extract_midi_features(path_df):
    """
    This function takes in the path DataFrame, then for each midi file, it extracts certain
    features, maps the genre to a number and concatenates these to a large design matrix to return.
    
    @input path_df: A dataframe with paths to midi files, as well as their corresponding matched genre.
    @type path_df: pandas.DataFrame
    
    @return: A matrix of features along with label.
    @rtype: numpy.ndarray of float
    """
    all_features = []
    for index, row in path_df.iterrows():
        features = get_features(row.Path)
        genre = label_dict[row.Genre]
        if features is not None:
            features.append(genre)
            all_features.append(features)
    return np.array(all_features)

labeled_features = extract_midi_features(matched_midi_df)
print(labeled_features)

[[ 0.15523849  0.55        0.125       0.125       1.        ]
 [ 0.15523849  0.55        0.125       0.125       1.        ]
 [ 0.06335386 -0.17        0.125       0.125       0.        ]
 ...
 [ 0.43089736  0.55        0.125       0.125       1.        ]
 [ 0.4333338  -0.35        0.125       0.125       1.        ]
 [ 0.41467545  1.91        0.125       0.125       1.        ]]
CPU times: user 52min 7s, sys: 1min 21s, total: 53min 29s
Wall time: 56min 4s


In [7]:
# Shuffle Entire Dataset to Make Random
labeled_features = np.random.permutation(labeled_features)

# Partition into 3 Sets
num = len(labeled_features)
num_training = int(num * 0.6)
num_validation = int(num * 0.8)
training_data = labeled_features[:num_training]
validation_data = labeled_features[num_training:num_validation]
test_data = labeled_features[num_validation:]

# Separate Features from Labels
num_cols = training_data.shape[1] - 1
training_features = training_data[:, :num_cols]
validation_features = validation_data[:, :num_cols]
test_features = test_data[:, :num_cols]

# Format Features for Multi-class Classification
num_classes = len(label_list)
training_labels = training_data[:, num_cols].astype(int)
validation_labels = validation_data[:, num_cols].astype(int)
test_labels = test_data[:, num_cols].astype(int)

# Function for One-Hot Encoding
def one_hot(labels):
    """
    This function encodes the labels using one-hot encoding.
    
    @input num_classes: The number of genres/classes.
    @type num_classes: int
    @input labels: The genre labels to encode.
    @type labels: numpy.ndarray of int
    
    @return: The one-hot encoding of the labels.
    @rtype: numpy.ndarray of int
    """
    return np.eye(num_classes)[labels].astype(int)

# Print to Check Dimentions and to Visualize
print(test_features[:10])
print(test_labels[:10])
print(one_hot(test_labels)[:10])

[[ 0.16666667  0.55        0.125       0.125     ]
 [ 0.24169193  0.31        0.125       0.125     ]
 [ 0.2140467  -0.35        0.125       0.125     ]
 [ 0.1556701  -0.35        1.125       0.625     ]
 [ 0.30061672 -0.35        0.125       0.125     ]
 [-0.02216318 -0.4325      0.125       0.125     ]
 [ 0.12371831 -0.35        0.125       0.125     ]
 [ 0.14516129 -0.35        0.125       0.125     ]
 [ 0.02754932  0.55        0.125       0.125     ]
 [ 0.34615646 -0.17        0.125       0.125     ]]
[ 1 11  8  1 11  1  1 10  1 11]
[[0 1 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 1 0]
 [0 0 0 0 0 0 0 0 1 0 0 0 0]
 [0 1 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 1 0]
 [0 1 0 0 0 0 0 0 0 0 0 0 0]
 [0 1 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 1 0 0]
 [0 1 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 1 0]]


In [8]:
def train_model(t_features, t_labels, v_features, v_labels):
    """
    This function trains a neural network using a couple different configurations.
    
    @input t_features: The training features.
    @type t_features: numpy.ndarray of float
    @input t_labels: The training labels.
    @type t_labels: numpy.ndarray of int
    @input v_features: The validation features.
    @type v_features: numpy.ndarray of float
    @input v_labels: The validation labels.
    @type v_labels: numpy.ndarray of int
    
    @return: The classifier that achieved the best validation accuracy.
    @rtype: sklearn.neural_network.multilayer_perceptron.MLPClassifier
    """
    # Neural Network and SVM Configurations
    clf_1 = MLPClassifier(solver='adam', alpha=1e-4, hidden_layer_sizes=(5,), random_state=1)
    clf_2 = MLPClassifier(solver='adam', alpha=1e-4, hidden_layer_sizes=(5, 5), random_state=1)
    clf_3 = MLPClassifier(solver='adam', alpha=1e-5, hidden_layer_sizes=(10, 10), random_state=1)
    clf_4 = MLPClassifier(solver='adam', alpha=1e-5, hidden_layer_sizes=(100, 100), random_state=1)
    clf_svm = SVC()
    
    # Keep Track of the Best Model
    best_clf = None
    best_accuracy = 0
    
    # Test the Accuracies of the Models and Get Best
    for clf in [clf_1, clf_2, clf_3, clf_4, clf_svm]:
        t_labels_hot = one_hot(t_labels)
        v_labels_hot = one_hot(v_labels)
        if (type(clf) == SVC):
            clf = clf.fit(t_features, t_labels)
        else:
            clf = clf.fit(t_features, t_labels_hot)
        predictions = clf.predict(v_features)
        count = 0
        for i in range(len(v_labels)):
            if (type(clf) != SVC):
                if np.array_equal(v_labels_hot[i], predictions[i]):
                    count += 1
            else:
                if v_labels[i] == predictions[i]:
                    count += 1
        accuracy = count / len(v_labels_hot)
        if accuracy > best_accuracy:
            best_accuracy = accuracy
            best_clf = clf

    print("Best Accuracy:", best_accuracy)
    return best_clf

classifier = train_model(training_features, training_labels, validation_features, validation_labels)

/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


Best Accuracy: 0.7392172523961661


In [9]:
def calculate_accuracy(clf, t_features, t_labels):
    """
    This function takes a trained model as well as the test features and its
    corresponding labels, and reports the accuracy of the model.
    
    @input clf: The trained classifier.
    @type model: sklearn.neural_network.multilayer_perceptron.MLPClassifier
    @input t_features: The features from the test set.
    @type f_features: numpy.ndarray of float
    @input t_labels: The labels of the test set features.
    @type t_labels: numpy.ndarray of int
    
    @return: The accuracy.
    @rtype: float
    """
    count = 0
    predictions = clf.predict(t_features)
    t_labels_hot = one_hot(t_labels)
    for i in range(len(t_features)):
        if (type(clf) == SVC):
            if t_labels[i] == predictions[i]:
                count += 1
        else:
            if np.array_equal(t_labels_hot[i], predictions[i]):
                count += 1
    return count / len(t_features)

# Print the Test Accuracy
print(calculate_accuracy(classifier, test_features, test_labels))

0.7321357285429142


In [10]:
def make_prediction(clf, midi_path):
    """
    This function uses the classifier to predict the genre of a midi file.
    
    @input clf: The trained classifier.
    @type clf: sklearn.neural_network.multilayer_perceptron.MLPClassifier
    @input midi_path: The path to the midi file that we are trying to classify.
    @type midi_path: String
    
    @return: The predicted genre of the midi file.
    @rtype: String
    """
    features = get_features(midi_path)
    prediction_ind = list(clf.predict([features])[0]).index(1)
    prediction = label_list[prediction_ind]
    return prediction
    
# Make a Prediction
test_midi_path ="lmd_matched/B/F/E/TRBFELB128F426BFF2/289270d85c81802d912c9907c645dc2d.mid"
print(make_prediction(classifier, test_midi_path))

Pop_Rock


In [25]:
output = "./output_midi/base_model_30_bars/from_scratch/16.midi"
print(make_prediction(classifier, output))

Pop_Rock
